In [ ]:
# Google Colab에서 Google 드라이브를 "연결"하는 역할을 해주는 코드 
"드라이브에서 데이터(책)를 꺼내와 Colab(책상)에서 작업하려는" 과정

In [2]:
# 모델 분석 및 시각화 도구
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import xml.etree.ElementTree as ET
import os
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# 모델 설계
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, Flatten, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import numpy as np
from PIL import Image

# 데이터셋 경로
dataset_path = "/content/drive/MyDrive/Dataset"

# 클래스와 라벨 매핑
classes = {"healthy_strawberry": 0, "infected_strawberry": 1}

# 데이터와 라벨 저장
image_data = []
labels = []

# 폴더를 순회하며 이미지 로드 및 라벨링
for class_name, label in classes.items():
    class_folder = os.path.join(dataset_path, class_name)
    for image_file in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_file)

        # 이미지 로드 (필요에 따라 리사이즈)
        try:
            img = Image.open(image_path).convert("RGB")  # RGB로 변환
            img = img.resize((128, 128))  # 이미지 크기 변경 (예: 128x128)
            image_data.append(np.array(img))
            labels.append(label)
        except Exception as e:
            print(f"이미지 로드 실패: {image_path}, 오류: {e}")

# 배열로 변환
X = np.array(image_data)
y = np.array(labels)

# 라벨 분포 출력
print("라벨 분포:", dict(zip(*np.unique(y, return_counts=True))))


라벨 분포: {0: 1015, 1: 976}
결과 해석: 1015: 건강한 딸기의 이미지 개수 / 976: 감염된 딸기의 이미지 개수/ 총 데이터 개수: 1015+976=1991개.
두 클래스의 데이터 비율이 약 51:49로 균형에 가까움
클래스 불균형 문제 없이 모델을 훈련하기에 적합한 데이터셋



1. 딸기 이미지 데이터셋을 처리하여 머신러닝 모델 학습에 사용할 준비를 하는 코드로 딸기 이미지가 건강한 딸기인지 감염된
딸기인지를 분류하기 위해 데이터를 로드, 전처리, 라벨링하는 과정
=> 다양한 크기와 색깔을 가진 이미지를 스캔하여, 같은 크기의 디지털 표지로 변환하고, 각 표지에 라벨(건강/감염) 번호를 붙이는 과정

데이터 셋 = 학습셋 + 검증셋 + 테스트셋

2. "healthy_strawberry": 0 (건강한 딸기) "infected_strawberry": 1 (감염된 딸기)
3. 전처리: 이미지를 RGB로 변환하고 크기를 (128, 128)로 조정
4. 데이터 저장: 변환된 이미지를 image_data에 라벨을 labels에 추가 



초기 상태에서 건강한 딸기는 1015개, 감염된 딸기는 976개로 이미 건강한 딸기가 더 많음
그런데 균형 잡힌 데이터셋을 만들기 위해, 두 클래스의 개수가 비슷해져야 함.

=> 왜 건강한 딸기를 증강 시킴?
모델이 학습 할 때, 건강한딸기와 감염된 딸기 사이의 비율 차이가 있으면 더 많이 주어진 클래스에 대한 예측을 우선적으로 
잘하게 됨, 감염된 딸기에 대해서는 덜 정확한 예측을 하게 될 가능성이 높아짐

==> 건강한 딸기가 더 많다고 해서 감염된 딸기를 증강하는 것이 더 적합할 수도 있지만, 여기서 중요한 점은, 감염된 딸기 이미지는 이미 적은 수이기 때문에
건강한 딸기를 증강시켜서 균형을 맞추는 방법을 선택한 것



from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
from sklearn.utils import shuffle
import numpy as np

# 데이터 증강 객체
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# healthy_strawberry와 infected_strawberry 이미지를 분리
healthy_images = X[y == 0]  # healthy_strawberry
infected_images = X[y == 1]  # infected_strawberry

augmented_images = []
augmented_labels = []

# 'healthy_strawberry' 이미지를 증강하여 'infected_strawberry' 개수에 맞추기
for image in healthy_images:
    image = image.reshape((1, *image.shape))  # (1, 128, 128, 3)로 변환
    for batch in datagen.flow(image, batch_size=1):
        augmented_images.append(batch[0])
        augmented_labels.append(0)  # healthy_strawberry 라벨
        if len(augmented_images) >= len(infected_images):  # infected_strawberry 개수만큼 증강
            break

# 증강된 이미지를 numpy 배열로 변환
augmented_images = np.array(augmented_images, dtype=np.uint8)
augmented_labels = np.array(augmented_labels)

# 원본 이미지와 증강된 이미지를 합침
X_balanced = np.concatenate([X, augmented_images], axis=0)
y_balanced = np.concatenate([y, augmented_labels], axis=0)

# 데이터 섞기
X_balanced, y_balanced = shuffle(X_balanced, y_balanced, random_state=42)

# 데이터 증강 이후 라벨 분포 확인
print("데이터 증강 이후 밸런스 확인:", dict(zip(*np.unique(y_balanced, return_counts=True))))

# 증강된 이미지 몇 개를 출력
plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    # BGR -> RGB 변환
    image = augmented_images[i]
    plt.imshow(image)  # 이제 이미지는 RGB로 출력됩니다.
    plt.title("healthy_strawberry (augmented_images)")
    plt.axis('off')
plt.tight_layout()
plt.show()


실행 결과 : 데이터 증강 이후 밸런스 확인: {0: 3005, 1: 976}

건강한 딸기 이미지가 3,005장이고, 감염된 딸기 이미지가 976장

건강한 딸기(healthy_strawberry) 이미지의 개수를 감염된 딸기(infected_strawberry) 이미지의 
개수에 맞추기 위해 건강한 딸기 이미지를 증강

데이터 증강 후 건강한 딸기의 개수는 3005개로 증가하고, 감염된 딸기는 976개로 그대로 유지

1. 필요한 라이브러리 임포트
2. 데이터 증강 설정
3. 데이터 분리 
4. 증강된 이미지와 라벨을 저장할 리스트 초기화
5.  'healthy_strawberry' 이미지 증강
if len(augmented_images) >= len(infected_images): 
    infected_strawberry 이미지 개수만큼 증강하면 중단.
6. 증강된 데이터를 배열로 변환
7. 원본 데이터와 증강 데이터 합치기 
8. 데이터 섞기
9. 증강 후 데이터 라벨 분포 확인
10. 증강된 이미지 시각화 

1. 손실 함수와 활성화 함수 
이진 분류(binary classification) 문제에서는 손실 함수로 binary_crossentropy를 사용
출력값이 0 또는 1인 경우에 적합
다중 클래스 분류(multi-class classification) 문제에서는 softmax 활성화 함수를 사용

2. 데이터셋 구성
학습셋(X_train, y_train): 모델 학습에 사용.
검증셋(validation_data): 학습 중 성능 평가용. 시험 전에 푸는 모의고사와 유사.
테스트셋(X_test, y_test): 최종 평가용. 학습에 포함되지 않음. 수능처럼 한 번의 평가로 성능을 확인.
검증셋을 꼭 나눌 필요는 없으며, 사용 목적에 따라 학습셋에서 분리하거나 대체 기법을 사용


#model_single.predict(X_test)는 테스트 데이터셋 X_test에 대해 모델이 예측한 확률을 반환
확률은 각 클래스(healthy_strawberry 또는 infected_strawberry)에 대한 소속 확률
#확률이 0.5보다 크면 클래스 1(감염된 딸기), 그렇지 않으면 클래스 0(건강한 딸기)으로 예
astype(int)는 예측값을 0 또는 1의 정수형으로 변환
#accuracy_score(y_test, y_pred_classes)는 모델의 예측값(y_pred_classes)과 실제 값(y_test)을 비교하여 정확도를 계산
정확도는 전체 예측 중 맞힌 비율을 나타냄ㅁ.
#plot_history 함수는 학습 중에 기록된 손실값(loss)과 정확도(accuracy)를 시각화하여 학습 과정을 확인할 수 있게 도와

4. 모델학습 과정
X_train, y_train: 학습 데이터 (문제와 정답).
epochs=50: 학습 반복 횟수. 데이터를 처음부터 끝까지 총 50번 학습.
batch_size=16: 데이터를 작은 묶음으로 나누어 처리.
너무 작으면 학습이 불안정(오버피팅 위험), 너무 크면 학습 속도 저하.
validation_data: 검증셋(X_test, y_test)을 학습 중 성능 평가에 사용.

5. 조기 종료: 성능 개선이 멈출 경우 학습을 자동으로 중단하여 시간을 절약 
monitor='val_loss': 검증 손실(val_loss)을 관찰.
patience=10: 성능이 개선되지 않아도 10 에포크 더 대기.
restore_best_weights: 최적의 가중치로 복원.

6. 오버피팅 방지 방법 //오버피팅: 학습셋에서만 성능이 높고 새로운 데이터에서 성능이 떨어지는 현상

1.드롭아웃(Dropout): 일부 뉴런을 무작위로 비활성화.
2.배치 정규화(Batch Normalization): 데이터 분포를 일정하게 유지.
3.활성화 함수 지정: 은닉층에 relu를 주로 사용.
4.은닉층 증가: 모델 복잡도를 적절히 늘림.

6. 정밀도: 모델이 예측한 양성 중 실제 양성 비율.
   재현율: 실제 양성 중 모델이 정확히 예측한 비율.

   #confusion_matrix(y_test, y_pred_classes)는 실제 값(y_test)과 예측값(y_pred_classes)을 비교하여 **혼동 행렬(confusion matrix)**을 계산
혼동 행렬은 예측이 잘못된 경우와 맞춘 경우를 나타내는 표
예측된 클래스와 실제 클래스의 분포를 확인할 수 있음.

정확도: 모델이 전체 데스트 데이터에서 얼마나 정확한 예측을 했는지 보여주고, 
혼동행렬: 각 클래스별로 예측이 잘 되었는지 - 잘못되었는지를 나타내며
분류 리포트: 모델의 세부적인 성능 지표(정밀도, 재현율 등)를 제공 
=> 모델의 예측 성능을 평가하고 모델이 건강한 딸기와 감염된 딸기를 얼마나 잘 구별하는지 확인하는 데 도움